# LCL test

In [1]:
import metpy.calc as calc
import metpy._calc_mod as calc_mod
from metpy.units import units
import numpy as np

In [2]:
shape = (50, 800, 800)

# Generate synthetic temperature (in degC) with a realistic gradient
T_C = np.random.normal(loc=25.0, scale=10.0, size=shape)  # e.g., mean 25°C, stddev 10

# Generate dew point temperatures a few degrees lower than temperature
dewT_C = T_C - np.abs(np.random.normal(loc=3.0, scale=2.0, size=shape))  # always <= T_C

# Generate pressure values in hPa, decreasing with altitude
z = np.linspace(0, 8, shape[0])  # approx. 0 to 12 km altitude
P_levels = 1013.25 * np.exp(-z / 7.0)  # simple exponential decay
P_hPa = P_levels[:, None, None] * np.ones((1, shape[1], shape[2]))

In [4]:
# save the data
np.savez('data.npz', T_C=T_C, dewT_C=dewT_C, P_hPa=P_hPa)

In [2]:
# load the data
data = np.load('data.npz')

T_C = data['T_C']
dewT_C = data['dewT_C']
P_hPa = data['P_hPa']

## Original LCL in Python

In [5]:
p_lcl, t_lcl = calc.lcl(P_hPa * units.hPa, T_C * units.degC, dewT_C * units.degC)

/var/folders/f3/vr_02qzd5snc7zk0m7cmwmy80000gn/T/ipykernel_8367/139428514.py:2: UserWarning: Saturation mixing ratio is undefined for some requested pressure/temperature combinations. Total pressure must be greater than the water vapor saturation pressure for liquid water to be in equilibrium.
  p_lcl, t_lcl = calc.lcl(P_hPa * units.hPa, T_C * units.degC, dewT_C * units.degC)


#### time used: 11 s

In [7]:
idx_nan = np.argwhere(np.isnan(p_lcl))
idx_nan

array([[ 40,  32, 504],
       [ 44, 731,  33],
       [ 46,  28, 374],
       [ 49, 754, 546]])

In [8]:
print("T: ", T_C[40,  32, 504], "degC")
print("dewT: ", dewT_C[40,  32, 504], "degC")
print("P: ", P_hPa[40,  32, 504], "hPa")

T:  77.64458546365549 degC
dewT:  76.46342469872707 degC
P:  398.60607922080777 hPa


## LCL manual vectorization in Python

In [11]:
calc.lcl_manual(P_hPa * units.hPa, T_C * units.degC, dewT_C * units.degC)

Warning in function 'SaturationMixingRatio': Total pressure must be greater than the saturation vapor pressure for liquid water to be in equilibrium.
Warning in function 'lambert_wm1': lambert_wm1 did not converge.
Warning in function 'SaturationMixingRatio': Total pressure must be greater than the saturation vapor pressure for liquid water to be in equilibrium.
Warning in function 'lambert_wm1': lambert_wm1 did not converge.
Warning in function 'SaturationMixingRatio': Total pressure must be greater than the saturation vapor pressure for liquid water to be in equilibrium.
Warning in function 'lambert_wm1': lambert_wm1 did not converge.
Warning in function 'SaturationMixingRatio': Total pressure must be greater than the saturation vapor pressure for liquid water to be in equilibrium.
Warning in function 'lambert_wm1': lambert_wm1 did not converge.


(<Quantity([[[ 922.88373239  987.72926347  971.76229072 ...  940.60128723
     976.40022727  949.43666988]
   [ 972.65574864  930.91846973  928.66070841 ...  944.13425675
     957.73973679  976.67920928]
   [ 997.69787307  985.22053275  922.64071162 ... 1001.27328924
     982.21872811  967.27923298]
   ...
   [1007.41406607  972.08610407  989.60009558 ...  987.68912927
     984.91978178  982.60990512]
   [ 960.4361106   956.76029394  988.40542749 ...  993.78392528
     989.26573622  933.85869847]
   [ 956.79452247  930.56431947 1005.60998252 ...  952.27959916
     970.06683537  961.5559487 ]]
 
  [[ 936.04857022  948.1215181   966.83395754 ...  951.78283821
     986.70250655  982.83830772]
   [ 914.22785428  976.96552053  940.70686194 ...  960.34508049
     936.0339513   950.29648823]
   [ 937.70162913  979.53209128  948.40589656 ...  953.68753113
     956.95848757  986.61561128]
   ...
   [ 964.91682235  964.44544765  928.0152724  ...  981.37518239
     955.2062229   939.69126183]
   

#### time used: 21 s

## LCL manual vectorization in c++ pybind11

In [3]:
p_lcl, t_lcl = calc_mod.lcl(P_hPa * 100, T_C+273.15, dewT_C+273.15)

Warning in function 'SaturationMixingRatio': Total pressure must be greater than the saturation vapor pressure for liquid water to be in equilibrium.
Warning in function 'lambert_wm1': lambert_wm1 did not converge.
Warning in function 'SaturationMixingRatio': Total pressure must be greater than the saturation vapor pressure for liquid water to be in equilibrium.
Warning in function 'lambert_wm1': lambert_wm1 did not converge.
Warning in function 'SaturationMixingRatio': Total pressure must be greater than the saturation vapor pressure for liquid water to be in equilibrium.
Warning in function 'lambert_wm1': lambert_wm1 did not converge.
Warning in function 'SaturationMixingRatio': Total pressure must be greater than the saturation vapor pressure for liquid water to be in equilibrium.
Warning in function 'lambert_wm1': lambert_wm1 did not converge.


#### time used: 6 s